# 機械学習のためのSQL

機械学習では、モデル選択より前にデータを正しく整形することが重要です。
実務ではその整形の多くを SQL で行います。SQL で作った特徴量は再現しやすく、学習環境と本番環境の差も小さくできます。

このノートでは、ECサービスの行動データを題材に、SQL の基本操作から、リークを避けた特徴量テーブル作成までを一気通貫で確認します。

In [ ]:
import sqlite3
from datetime import datetime, timedelta
from random import Random
from textwrap import dedent


まずはメモリ上にデータベースを作り、`users`, `events`, `orders` の3テーブルを定義します。
`events` は行動ログ、`orders` は購入履歴、`users` は属性情報です。

In [ ]:
conn = sqlite3.connect(":memory:")
conn.row_factory = sqlite3.Row
cur = conn.cursor()

cur.executescript(
    dedent(
        """
        CREATE TABLE users (
            user_id INTEGER PRIMARY KEY,
            signup_date TEXT NOT NULL,
            plan TEXT NOT NULL,
            country TEXT NOT NULL
        );

        CREATE TABLE events (
            event_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            event_time TEXT NOT NULL,
            event_type TEXT NOT NULL,
            session_seconds INTEGER NOT NULL,
            clicks INTEGER NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        );

        CREATE TABLE orders (
            order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            order_time TEXT NOT NULL,
            amount REAL NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        );
        """
    )
)
conn.commit()


In [ ]:
def q(sql: str, params=()):
    return conn.execute(dedent(sql), params).fetchall()


def show(rows, limit=8):
    rows = list(rows)
    if not rows:
        print("(no rows)")
        return
    cols = rows[0].keys()
    print(" | ".join(cols))
    print("-" * 100)
    for r in rows[:limit]:
        print(" | ".join(str(r[c]) for c in cols))
    if len(rows) > limit:
        print(f"... ({len(rows)} rows total)")


以降は次の2つの関数を使います。

- `q(sql, params)`: SQLを実行して結果行を返す
- `show(rows)`: 結果を見やすく表示する

例: `show(q("SELECT COUNT(*) AS n FROM users"))`

次に、分析用の疑似データを生成します。セルを3つに分けて、
`ユーザー生成 → イベント生成 → 注文生成` の順で作ります。

In [ ]:
rng = Random(42)
start = datetime(2024, 1, 1)
end = datetime(2024, 6, 30)

plans = ["free", "pro", "team"]
plan_weights = [0.62, 0.30, 0.08]
countries = ["JP", "US", "IN", "DE", "FR"]

n_users = 320
for user_id in range(1, n_users + 1):
    signup = start + timedelta(days=rng.randint(0, 80))
    plan = rng.choices(plans, weights=plan_weights, k=1)[0]
    country = rng.choice(countries)
    cur.execute(
        "INSERT INTO users (user_id, signup_date, plan, country) VALUES (?, ?, ?, ?)",
        (user_id, signup.date().isoformat(), plan, country),
    )

conn.commit()


In [ ]:
for row in q("SELECT user_id, plan, signup_date FROM users"):
    user_id = row["user_id"]
    plan = row["plan"]
    signup_dt = datetime.fromisoformat(row["signup_date"])

    base_events = 20 if plan == "free" else 34 if plan == "pro" else 46
    n_events = max(8, int(rng.gauss(base_events, 6)))

    for _ in range(n_events):
        day_offset = rng.randint(0, 178)
        event_dt = signup_dt + timedelta(days=day_offset, hours=rng.randint(0, 23), minutes=rng.randint(0, 59))
        if event_dt > end:
            continue

        if plan == "free":
            clicks = max(0, int(rng.gauss(2.2, 1.4)))
            session_seconds = max(20, int(rng.gauss(130, 70)))
        elif plan == "pro":
            clicks = max(0, int(rng.gauss(4.8, 2.0)))
            session_seconds = max(30, int(rng.gauss(220, 90)))
        else:
            clicks = max(0, int(rng.gauss(6.5, 2.4)))
            session_seconds = max(40, int(rng.gauss(290, 110)))

        event_type = rng.choices(
            ["page_view", "search", "add_to_cart"],
            weights=[0.62, 0.25, 0.13],
            k=1,
        )[0]

        cur.execute(
            """
            INSERT INTO events (user_id, event_time, event_type, session_seconds, clicks)
            VALUES (?, ?, ?, ?, ?)
            """,
            (user_id, event_dt.isoformat(sep=" "), event_type, session_seconds, clicks),
        )

conn.commit()


In [ ]:
activity = q(
    """
    SELECT
        u.user_id,
        u.plan,
        u.signup_date,
        COALESCE(COUNT(e.event_id), 0) AS event_count,
        COALESCE(SUM(e.clicks), 0) AS total_clicks
    FROM users u
    LEFT JOIN events e ON u.user_id = e.user_id
    GROUP BY u.user_id, u.plan, u.signup_date
    """
)

for r in activity:
    user_id = r["user_id"]
    plan = r["plan"]
    signup_dt = datetime.fromisoformat(r["signup_date"])
    total_clicks = r["total_clicks"]
    event_count = r["event_count"]

    base_prob = 0.02
    if plan == "pro":
        base_prob += 0.08
    if plan == "team":
        base_prob += 0.12
    base_prob += min(0.35, total_clicks / 380.0)
    base_prob += min(0.20, event_count / 420.0)

    n_orders = 0
    for _ in range(3):
        if rng.random() < base_prob:
            n_orders += 1

    min_order_dt = signup_dt + timedelta(days=1)
    max_order_dt = min(end, signup_dt + timedelta(days=180))
    if min_order_dt > max_order_dt:
        continue

    total_seconds = int((max_order_dt - min_order_dt).total_seconds())

    for _ in range(n_orders):
        offset_sec = rng.randint(0, total_seconds)
        order_dt = min_order_dt + timedelta(seconds=offset_sec)

        mean_amount = 38 if plan == "free" else 74 if plan == "pro" else 130
        amount = max(8, round(rng.gauss(mean_amount, mean_amount * 0.35), 2))

        cur.execute(
            "INSERT INTO orders (user_id, order_time, amount) VALUES (?, ?, ?)",
            (user_id, order_dt.isoformat(sep=" "), amount),
        )

conn.commit()


In [ ]:
table_counts = q(
    """
    SELECT 'users' AS table_name, COUNT(*) AS n FROM users
    UNION ALL
    SELECT 'events' AS table_name, COUNT(*) AS n FROM events
    UNION ALL
    SELECT 'orders' AS table_name, COUNT(*) AS n FROM orders
    """
)
show(table_counts)


ここから SQL の基本操作を確認します。
まずは `WHERE` と `ORDER BY` で、特定ユーザーのイベント履歴を時系列に取得します。

In [ ]:
rows = q(
    """
    SELECT user_id, event_time, event_type, session_seconds, clicks
    FROM events
    WHERE user_id = ?
    ORDER BY event_time
    LIMIT 10
    """,
    (12,),
)
show(rows)


次に `GROUP BY` でユーザー単位の集計を作ります。
この集計はそのまま特徴量の候補になります。

In [ ]:
agg = q(
    """
    SELECT
        user_id,
        COUNT(*) AS n_events,
        SUM(clicks) AS total_clicks,
        AVG(session_seconds) AS avg_session_seconds
    FROM events
    GROUP BY user_id
    HAVING COUNT(*) >= 18
    ORDER BY total_clicks DESC
    LIMIT 10
    """
)
show(agg)


`JOIN` と `CASE WHEN` を使うと、属性と行動を組み合わせた特徴量を作れます。
この例では、ある時点（`asof_ts`）より前90日間の購入金額を集計し、プランを数値化します。

In [ ]:
asof_ts = '2024-05-01 00:00:00'

joined = q(
    """
    WITH spend_90 AS (
      SELECT
          user_id,
          SUM(amount) AS spend_90d
      FROM orders
      WHERE order_time >= datetime(:asof_ts, '-90 day')
        AND order_time < :asof_ts
      GROUP BY user_id
    )
    SELECT
        u.user_id,
        u.plan,
        CASE WHEN u.plan = 'free' THEN 0 ELSE 1 END AS paid_flag,
        COALESCE(s.spend_90d, 0) AS spend_90d
    FROM users u
    LEFT JOIN spend_90 s ON u.user_id = s.user_id
    ORDER BY spend_90d DESC
    LIMIT 12
    """,
    {"asof_ts": asof_ts},
)
show(joined)


ここから機械学習用テーブルを作ります。

時点の定義:
- `snapshot`: 予測時点
- 特徴量窓: `snapshot` より前30日
- ラベル窓: `snapshot` 以上かつ `snapshot+30日` 未満

In [ ]:
snapshot = '2024-05-01 00:00:00'
horizon_days = 30

feature_sql = """
WITH event_30d AS (
    SELECT
        user_id,
        COUNT(*) AS events_30d,
        SUM(clicks) AS clicks_30d,
        AVG(session_seconds) AS avg_session_30d,
        MAX(event_time) AS last_event_time
    FROM events
    WHERE event_time < :snapshot
      AND event_time >= datetime(:snapshot, '-30 day')
    GROUP BY user_id
),
order_90d AS (
    SELECT
        user_id,
        SUM(amount) AS spend_90d
    FROM orders
    WHERE order_time < :snapshot
      AND order_time >= datetime(:snapshot, '-90 day')
    GROUP BY user_id
),
label_window AS (
    SELECT
        user_id,
        CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END AS purchased_30d
    FROM orders
    WHERE order_time >= :snapshot
      AND order_time < datetime(:snapshot, '+' || :horizon_days || ' day')
    GROUP BY user_id
)
SELECT
    u.user_id,
    u.country,
    CASE WHEN u.plan = 'free' THEN 0 ELSE 1 END AS paid_flag,
    COALESCE(e.events_30d, 0) AS events_30d,
    COALESCE(e.clicks_30d, 0) AS clicks_30d,
    COALESCE(e.avg_session_30d, 0) AS avg_session_30d,
    -- イベントがないユーザーは signup_date を最終行動日として扱う
    CAST((julianday(:snapshot) - julianday(COALESCE(e.last_event_time, u.signup_date))) AS INTEGER) AS days_since_last_event,
    COALESCE(o.spend_90d, 0) AS spend_90d,
    COALESCE(l.purchased_30d, 0) AS label
FROM users u
LEFT JOIN event_30d e ON u.user_id = e.user_id
LEFT JOIN order_90d o ON u.user_id = o.user_id
LEFT JOIN label_window l ON u.user_id = l.user_id
ORDER BY u.user_id
"""


In [ ]:
feature_rows = q(feature_sql, {"snapshot": snapshot, "horizon_days": horizon_days})
show(feature_rows, limit=12)
print('rows:', len(feature_rows))


In [ ]:
label_dist_sql = f"""
WITH base AS (
{feature_sql}
)
SELECT
    label,
    COUNT(*) AS n_users,
    ROUND(AVG(events_30d), 2) AS avg_events_30d,
    ROUND(AVG(spend_90d), 2) AS avg_spend_90d
FROM base
GROUP BY label
ORDER BY label
"""

label_dist = q(label_dist_sql, {"snapshot": snapshot, "horizon_days": horizon_days})
show(label_dist)


次に、リークの例を確認します。
比較を明確にするため、窓幅はどちらも30日に固定し、未来を含むかどうかだけを変えます。

In [ ]:
clean_stats_sql = f"""
WITH base AS (
{feature_sql}
)
SELECT label, ROUND(AVG(events_30d), 2) AS avg_events
FROM base
GROUP BY label
ORDER BY label
"""

leaky_stats_sql = """
WITH event_30d_leaky AS (
    SELECT
        user_id,
        COUNT(*) AS events_30d
    FROM events
    WHERE event_time >= :snapshot
      -- NG: snapshot以降（未来）のイベントを特徴量に含める
      AND event_time < datetime(:snapshot, '+' || :horizon_days || ' day')
    GROUP BY user_id
),
label_window AS (
    SELECT
        user_id,
        CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END AS label
    FROM orders
    WHERE order_time >= :snapshot
      AND order_time < datetime(:snapshot, '+' || :horizon_days || ' day')
    GROUP BY user_id
)
SELECT
    COALESCE(l.label, 0) AS label,
    ROUND(AVG(COALESCE(e.events_30d, 0)), 2) AS avg_events
FROM users u
LEFT JOIN event_30d_leaky e ON u.user_id = e.user_id
LEFT JOIN label_window l ON u.user_id = l.user_id
GROUP BY COALESCE(l.label, 0)
ORDER BY label
"""

clean_stats = q(clean_stats_sql, {"snapshot": snapshot, "horizon_days": horizon_days})
leaky_stats = q(leaky_stats_sql, {"snapshot": snapshot, "horizon_days": horizon_days})

print('clean feature mean by label')
show(clean_stats)
print('leaky feature mean by label')
show(leaky_stats)


最後に、学習用テーブルを分割できる形へ整えます。
`%` は余り演算子で、`x % 5` は 0〜4 の fold 番号を作ります。
ここでは分かりやすさ優先で `user_id % 5` を使います（実務ではハッシュ関数を使うことが多いです）。

In [ ]:
dataset_fold_sql = f"""
WITH base AS (
{feature_sql}
)
SELECT
    user_id,
    paid_flag,
    events_30d,
    clicks_30d,
    avg_session_30d,
    days_since_last_event,
    spend_90d,
    label,
    (user_id % 5) AS fold_id
FROM base
ORDER BY user_id
"""

dataset_with_fold = q(dataset_fold_sql, {"snapshot": snapshot, "horizon_days": horizon_days})
show(dataset_with_fold, limit=12)


SQL は機械学習パイプラインの前工程ではなく、学習品質を決める中核です。
特に重要なのは、ラベル時点と特徴量時点の境界を明示し、未来情報を混ぜないことです。

モデルを変える前に、SQL で作る学習テーブルが「本番時点でも再現可能か」を必ず確認してください。